In [1]:
%load_ext autoreload
%autoreload 2


import sys
import warnings


warnings.filterwarnings("ignore")
sys.path.append("../")

In [11]:
from modules.data.conll2003.prc import conll2003_preprocess

In [2]:
data_dir = "/home/eartemov/ae/work/conll2003/"

In [13]:
conll2003_preprocess(data_dir)

## IO markup

### Train

In [32]:
from modules.data import bert_data

In [4]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [5]:
from modules.models.bert_models import BERTBiLSTMCRF

In [6]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    lstm_dropout=0., crf_dropout=0.3)

In [7]:
from modules.train.train import NerLearner

In [8]:
num_epochs = 100

In [9]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-IO.cpt", t_total=num_epochs * len(data.train_dl))

In [10]:
model.get_n_trainable_params()

2235023

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [11]:
from modules.data import bert_data

In [12]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels.txt",
    clear_cache=True
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [13]:
from modules.models.bert_models import BERTBiLSTMCRF

In [14]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    lstm_dropout=0., crf_dropout=0.3)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmpcxh2myyl
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

In [15]:
from modules.train.train import NerLearner

In [16]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-IO.cpt")

In [17]:
model.get_n_trainable_params()

2235023

In [18]:
learner.load_model()

### Predict

In [19]:
from modules.data.bert_data import get_data_loader_for_predict

In [20]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [21]:
preds = learner.predict(dl)

In [22]:
from sklearn_crfsuite.metrics import flat_classification_report

In [23]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [24]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [25]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(true_labels, pred_labels, labels=data.train_ds.idx2label[4:], digits=4)

In [26]:
print(tokens_report)

              precision    recall  f1-score   support

       I_ORG     0.9502    0.9474    0.9488      2016
         I_O     0.9971    0.9970    0.9971     41702
      I_MISC     0.9249    0.9109    0.9178      1257
       I_PER     0.9836    0.9853    0.9844      2856
       I_LOC     0.9715    0.9564    0.9639      1926

   micro avg     0.9917    0.9906    0.9911     49757
   macro avg     0.9655    0.9594    0.9624     49757
weighted avg     0.9916    0.9906    0.9911     49757



In [28]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [29]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.9365    0.9197    0.9280      1668
        MISC     0.8979    0.8840    0.8909       905
           O     0.9919    0.9931    0.9925     41683
         ORG     0.8701    0.8619    0.8660      1282
         PER     0.9585    0.9596    0.9591      1684

    accuracy                         0.9837     47222
   macro avg     0.9310    0.9237    0.9273     47222
weighted avg     0.9836    0.9837    0.9836     47222



## BIO markup

### Train

In [11]:
from modules.data import bert_data

In [3]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels1.txt",
    clear_cache=True,
    markup="BIO"
)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.


In [4]:
from modules.models.bert_models import BERTBiLSTMCRF

In [5]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    lstm_dropout=0., crf_dropout=0.3)

In [6]:
from modules.train.train import NerLearner

In [7]:
num_epochs = 100

In [8]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [9]:
model.get_n_trainable_params()

2236155

In [ ]:
learner.fit(epochs=num_epochs)

### Eval

#### Load model

In [12]:
from modules.data import bert_data

In [13]:
data = bert_data.LearnData.create(
    train_df_path="/home/eartemov/ae/work/conll2003/eng.train.train.csv",
    valid_df_path="/home/eartemov/ae/work/conll2003/eng.testa.dev.csv",
    idx2labels_path="/home/eartemov/ae/work/conll2003/idx2labels1.txt",
    clear_cache=False,
    markup="BIO"
)

INFO:pytorch_pretrained_bert.tokenization:loading vocabulary file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased-vocab.txt from cache at /home/eartemov/.pytorch_pretrained_bert/96435fa287fbf7e469185f1062386e05a075cadbf6838b74da22bf64b080bc32.99bcd55fc66f4f3360bc49ba472b940b8dcf223ea6a345deb969d607ca900729


In [14]:
from modules.models.bert_models import BERTBiLSTMCRF

In [15]:
model = BERTBiLSTMCRF.create(
    len(data.train_ds.idx2label),
    lstm_dropout=0., crf_dropout=0.3)

INFO:pytorch_pretrained_bert.modeling:loading archive file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-multilingual-cased.tar.gz from cache at /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9
INFO:pytorch_pretrained_bert.modeling:extracting archive file /home/eartemov/.pytorch_pretrained_bert/731c19ddf94e294e00ec1ba9a930c69cc2a0fd489b25d3d691373fae4c0986bd.4e367b0d0155d801930846bb6ed98f8a7c23e0ded37888b29caa37009a40c7b9 to temp dir /tmp/tmplmqxuayi
INFO:pytorch_pretrained_bert.modeling:Model config {
  "attention_probs_dropout_prob": 0.1,
  "directionality": "bidi",
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "max_position_embeddings": 512,
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "

In [16]:
from modules.train.train import NerLearner

In [17]:
learner = NerLearner(
    model, data, "/home/eartemov/ae/work/models/conll2003-BERTBiLSTMCRF-BIO.cpt", t_total=num_epochs * len(data.train_dl))

In [18]:
model.get_n_trainable_params()

2236155

In [19]:
learner.load_model()

### Predict

In [20]:
from modules.data.bert_data import get_data_loader_for_predict

In [21]:
dl = get_data_loader_for_predict(data, df_path=data.valid_ds.config["df_path"])

In [22]:
preds = learner.predict(dl)

In [23]:
from sklearn_crfsuite.metrics import flat_classification_report

In [24]:
from modules.analyze_utils.utils import bert_labels2tokens, voting_choicer
from modules.analyze_utils.plot_metrics import get_bert_span_report

In [25]:
pred_tokens, pred_labels = bert_labels2tokens(dl, preds)
true_tokens, true_labels = bert_labels2tokens(dl, [x.bert_labels for x in dl.dataset])

In [28]:
assert pred_tokens == true_tokens
tokens_report = flat_classification_report(
    true_labels, pred_labels,
    labels=data.train_ds.idx2label[4:], digits=4)

In [29]:
print(tokens_report)

              precision    recall  f1-score   support

       B_ORG     0.9363    0.9509    0.9435      1282
         B_O     0.9969    0.9982    0.9976     41702
      B_MISC     0.9319    0.9072    0.9194       905
       B_PER     0.9792    0.9792    0.9792      1686
       I_PER     0.9922    0.9812    0.9867      1170
       B_LOC     0.9699    0.9658    0.9679      1669
       I_ORG     0.9574    0.9482    0.9528       734
      I_MISC     0.8980    0.8750    0.8863       352
       I_LOC     0.9751    0.9144    0.9438       257

    accuracy                         0.9912     49757
   macro avg     0.9597    0.9467    0.9530     49757
weighted avg     0.9912    0.9912    0.9912     49757



In [30]:
from modules.analyze_utils.plot_metrics import get_bert_span_report
from modules.analyze_utils.utils import voting_choicer, tokens2spans

In [31]:
print(get_bert_span_report(dl, preds, fn=voting_choicer))

              precision    recall  f1-score   support

         LOC     0.9270    0.9131    0.9200      1668
        MISC     0.8965    0.8707    0.8834       905
           O     0.9907    0.9921    0.9914     41681
         ORG     0.8769    0.8783    0.8776      1282
         PER     0.9487    0.9431    0.9459      1686

    accuracy                         0.9821     47222
   macro avg     0.9279    0.9194    0.9236     47222
weighted avg     0.9820    0.9821    0.9821     47222

